In [24]:
import os
import json
import time
import pickle
import argparse
import numpy as np
import pandas as pd
from utils.data_utils import getSimilar
from utils.make_pairs import generate_pairs_file
from utils.link_prediction import regression_prediction, load_word2vec_model, getlabelmap
import pandas as pd
import json
from urllib.parse import urlencode
import urllib.request, json

## Load the files 

compounds = 'data/compound_neighbors.csv'
compounds_url = 'http://chem2bio2rdf.org/pubchem/resource/pubchem_compound/'
targets = 'data/target.csv'
target_url = 'http://chem2bio2rdf.org/uniprot/resource/gene/'
nodes = 'data/nodes.txt'
data_folder = 'data/'
relations = 'compoundgenerelations.csv'
classifier = 'models/xgb.pkl'

In [25]:
def getAnumber(anum):
    
    smiles_url = "http://wa00475d:9944/auth/anon/launchjob?_compat=1&_protocol=Protocols%5CAbhik_TEST%5Cwebservice%5CA-Number_to_Smiles&avnumber="+anum
    print(smiles_url)

    with urllib.request.urlopen(smiles_url) as url:
        data = json.loads(url.read())
        return(data['Canonical_Smiles'])

# Load Similarity Index File    
fp_filename = '/home/ubuntu/jack/KnowledgeGraphAlgos/compound_gene_prediction/data/c2b2rdf.h5'

def getSimilar(query, top):
    params = {'smiles': query, 'top': top}
    qstr = urlencode(params)
    url = f'http://10.253.81.157:8181/getsimilar?{qstr}'
    response = requests.get(url)
    res = response.json()
    data = json.loads(res)
    df = pd.DataFrame(data)
    print(df)
    df['mol_id'].to_csv(compounds,header=None, index=False)
   # return(df['mol_id'])

In [30]:
## Input smiles get the top 4,5 similar compounds
import requests

## This is the imatinib put any other compound smiles
smiles = 'CC1(CCC(=C(C1)C2=CC=C(C=C2)Cl)CN3CCN(CC3)C4=CC(=C(C=C4)C(=O)NS(=O)(=O)C5=CC(=C(C=C5)NCC6CCOCC6)[N+](=O)[O-])OC7=CN=C8C(=C7)C=CN8)C'

## you can use A-number
## Predict using A-NUMBER 

## User webservices to get smiles from A-NUMBER
##anum = 'A-1195425.0'

#smiles = getAnumber(anum)
print(smiles)
getSimilar(smiles,1) ## 4 is the top 4 similar compound in chem2bio2


CC1(CCC(=C(C1)C2=CC=C(C=C2)Cl)CN3CCN(CC3)C4=CC(=C(C=C4)C(=O)NS(=O)(=O)C5=CC(=C(C=C5)NCC6CCOCC6)[N+](=O)[O-])OC7=CN=C8C(=C7)C=CN8)C
     mol_id     coeff
0  25139760  0.456522


In [27]:
pairs_file = os.path.join(data_folder, 'compound_gene_relations.csv')
pairs = generate_pairs_file(entity1=compounds, entity1_url=compounds_url, entity2=targets, entity2_url=target_url, nodes=nodes, output=pairs_file)
nodemap = getlabelmap(nodes)
n_nodes = len(nodemap)

--- Time to read files: 0.14975810050964355 seconds ---
found 295911 nodes in dict file


In [31]:
## select the model to run 

mymodel = "edge2vec"
output_fname = "./data/e2v_venetoclax.txt"

if mymodel == "edge2vec":
    
    vectors = 'models/edge2vec_vectors.npm'
    start = time.time()
    model = np.memmap(vectors, dtype='float32', mode='r')
    print(f'--- Time to load model ---\n {time.time()-start} seconds\n')
    shape = (n_nodes, len(model)//n_nodes)
    model = model.reshape(shape)

if mymodel == "node2vec":
    vectors = 'models/node2vec_vectors.npm'
    start = time.time()
    model = np.memmap(vectors, dtype='float32', mode='r')
    print(f'--- Time to load model ---\n {time.time()-start} seconds\n')

    shape = (n_nodes, len(model)//n_nodes)
    model = model.reshape(shape)

## Run the link prediction algorithm 
regression_prediction(model=model, eval_file=pairs, output=output_fname, nodemap=nodemap, load_model_path=classifier, model_type='edge2vec')

--- Time to load model ---
 0.003068685531616211 seconds

--- LOAD TEST SET ---
build instance 0
build instance 100
build instance 200
build instance 300
build instance 400
build instance 500
build instance 600
build instance 700
build instance 800
build instance 900
build instance 1000
build instance 1100
build instance 1200
build instance 1300
build instance 1400
build instance 1500
build instance 1600
build instance 1700
build instance 1800
build instance 1900
build instance 2000
build instance 2100
build instance 2200
build instance 2300
build instance 2400
build instance 2500
build instance 2600
build instance 2700
build instance 2800
build instance 2900
build instance 3000
build instance 3100
build instance 3200
build instance 3300
build instance 3400
build instance 3500
build instance 3600
build instance 3700
build instance 3800
build instance 3900
build instance 4000
build instance 4100
build instance 4200
build instance 4300
build instance 4400
build instance 4500
build instan

(11264,)


In [32]:
from jupyter_datatables import init_datatables_mode

init_datatables_mode()

pred = pd.read_csv(output_fname, sep='\t',header=None,engine='python')
new = pred.iloc[:,1].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
pred["score"] = pred[0]
pred["Compound"]= new[0] 
# making separate last name column from new data frame 
pred["Gene"]= new[1] 
pred.drop(pred.columns[[0,1,3]], axis=1,inplace=True)
pred["Gene"]= pred["Gene"].replace("http://chem2bio2rdf.org/uniprot/resource/gene/","",regex=True)
predicted = pred.groupby(['Gene'],as_index=False).mean().sort_values('score',ascending=False)
predicted

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

Gene     score
2094  PPP2R2A  0.999854
31      ACADS  0.999821
163    ANTXR1  0.999820
1552   MAP4K5  0.999806
2466  SLC22A5  0.999797
...       ...       ...
696       DBI  0.849982
1233     HTR6  0.843623
1332    ITGB4  0.781608
1912     PAK6  0.758542
2484  SLC36A1  0.638126

[2816 rows x 2 columns]